# Testing geocoding with OpenStreetMap

In [19]:
import urllib
import requests

# from https://www.natasshaselvaraj.com/a-step-by-step-guide-on-geocoding-in-python/
def geocode_response(locality):
    url = 'https://nominatim.openstreetmap.org/search/' + urllib.parse.quote(locality) +'?format=json'
    response = requests.get(url).json()
    if(len(response)!=0):
        return(response[0])
        #return(response[0]['lat'], response[0]['lon'])
    else:
        return('-1')

geocode_response('London,ON')

{'place_id': 283294785,
 'licence': 'Data © OpenStreetMap contributors, ODbL 1.0. https://osm.org/copyright',
 'osm_type': 'relation',
 'osm_id': 7485368,
 'boundingbox': ['42.8245667', '43.0730461', '-81.3906556', '-81.1070784'],
 'lat': '42.9832406',
 'lon': '-81.243372',
 'display_name': 'London, London (city), Southwestern Ontario, Ontario, Canada',
 'class': 'boundary',
 'type': 'administrative',
 'importance': 0.7759177286656098,
 'icon': 'https://nominatim.openstreetmap.org/ui/mapicons/poi_boundary_administrative.p.20.png'}

In [10]:
res = geocode_response('Toronto,ON')

res['display_name'].split(',')[-1].strip()

'Canada'

# Extracting Hashtags, Language, Country, and Timestamp from a raw CSV

In [11]:
import os
import sys
import pandas as pd

raw_csv_files_folder_path = "..\\data\\archive\\" # up Jupyter\ Folder, into data\archive\

raw_csv_filepaths = [os.path.join(raw_csv_files_folder_path, f) for f in os.listdir(raw_csv_files_folder_path) if f.endswith('.csv')]

# experiment with the first csv

raw_csv_filepath = raw_csv_filepaths[0]

df = pd.read_csv(raw_csv_filepath)

df.info()

C:\Users\Marcus\AppData\Local\Temp\ipykernel_6648\2772176729.py:13: DtypeWarning: Columns (15) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(raw_csv_filepath)


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 364875 entries, 0 to 364874
Data columns (total 18 columns):
 #   Column          Non-Null Count   Dtype 
---  ------          --------------   ----- 
 0   Unnamed: 0      364875 non-null  int64 
 1   userid          364875 non-null  int64 
 2   username        364875 non-null  object
 3   acctdesc        286432 non-null  object
 4   location        212933 non-null  object
 5   following       364875 non-null  int64 
 6   followers       364875 non-null  int64 
 7   totaltweets     364875 non-null  int64 
 8   usercreatedts   364875 non-null  object
 9   tweetid         364875 non-null  int64 
 10  tweetcreatedts  364875 non-null  object
 11  retweetcount    364875 non-null  int64 
 12  text            364875 non-null  object
 13  hashtags        364875 non-null  object
 14  language        364875 non-null  object
 15  coordinates     97 non-null      object
 16  favorite_count  364875 non-null  int64 
 17  extractedts     364875 non-nu

In [12]:
df.head(5)

,Unnamed: 0,userid,username,acctdesc,location,following,followers,totaltweets,usercreatedts,tweetid,tweetcreatedts,retweetcount,text,hashtags,language,coordinates,favorite_count,extractedts
0,0,16882774,Yaniela,"Animal lover, supports those who fight injusti...",Hawaii,1158,392,88366,2008-10-21 07:34:04.000000,1509681950042198030,2022-04-01 00:00:00.000000,3412,⚡The Ukrainian Air Force would like to address...,[],en,NaN,0,2022-04-01 00:44:20.097867
1,1,3205296069,gregffff,NaN,NaN,122,881,99853,2015-04-25 11:24:34.000000,1509681950151348229,2022-04-01 00:00:00.000000,100,Chernihiv oblast. Ukrainians welcome their lib...,"[{'text': 'russianinvasion', 'indices': [77, 9...",en,NaN,0,2022-04-01 00:09:37.148770
2,2,1235940869812809728,ThanapornThon17,เล่นไวโอลิน\nพูดภาษาจีน,NaN,231,72,5481,2020-03-06 14:52:01.000000,1509681950683926556,2022-04-01 00:00:00.000000,9,America 🇺🇸 is preparing for something worse th...,"[{'text': 'RussianUkrainianWar', 'indices': [7...",en,NaN,0,2022-04-01 00:09:37.125654
3,3,1347985375566966784,I_Protest_2021,01000001 01101110 01101111 01101110 01111001 0...,International Web Zone,399,377,301,2021-01-09 19:15:44.000000,1509681951116046336,2022-04-01 00:00:00.000000,573,JUST IN: #Anonymous has hacked &amp; released ...,"[{'text': 'Anonymous', 'indices': [25, 35]}]",en,NaN,0,2022-04-01 00:31:39.674382
4,4,1505394816636846083,Marsh_Win_01,🌿@Pickaw @TWITTERPICKER 🌿Winning isn’t everyth...,Hunter Account,158,25,8982,2022-03-20 04:04:40.000000,1509681951304990720,2022-04-01 00:00:00.000000,190,***PUBLIC MINT NOW LIVE***\n\nFor \n@billionai...,[],en,NaN,0,2022-04-01 00:09:37.101459


In [13]:
df = df[['tweetid','hashtags','language','location']]

df.dropna(inplace=True) # drop NaN in location column

df = df[df['hashtags'].map(lambda d: d != '[]')] # drop rows with empty hashtags

import datetime

# from https://ws-dl.blogspot.com/2019/08/2019-08-03-tweetedat-finding-tweet.html
def get_tweet_timestamp(tid):
    offset = 1288834974657 # UTC offset in milliseconds
    tstamp = (tid >> 22) + offset
    return datetime.datetime.utcfromtimestamp(tstamp/1000)

df['tweet_timestamp'] = df['tweetid'].map(get_tweet_timestamp)

import json

def simplify_hashtags(htag_json):
    htag_json = htag_json.replace('\'','\"')
    return [ht['text'].lower() for ht in json.loads(htag_json)]

df['hashtags'] = df['hashtags'].map(simplify_hashtags)

df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 171399 entries, 3 to 364873
Data columns (total 5 columns):
 #   Column           Non-Null Count   Dtype         
---  ------           --------------   -----         
 0   tweetid          171399 non-null  int64         
 1   hashtags         171399 non-null  object        
 2   language         171399 non-null  object        
 3   location         171399 non-null  object        
 4   tweet_timestamp  171399 non-null  datetime64[ns]
dtypes: datetime64[ns](1), int64(1), object(3)
memory usage: 7.8+ MB


In [14]:
df.head(5)

,tweetid,hashtags,language,location,tweet_timestamp
3,1509681951116046336,[anonymous],en,International Web Zone,2022-04-01 00:00:00.337
7,1509681953053843466,"[russian, moscow]",en,India,2022-04-01 00:00:00.799
9,1509681953418711050,"[putin, medvedev, russia, ukraine]",en,"Kurdistan, Worldwide",2022-04-01 00:00:00.886
10,1509681953418752008,"[russia, putin]",en,Dubai,2022-04-01 00:00:00.886
11,1509681953636700160,"[russianarmy, chernobyl, russianukrainianwar]",en,"Ontario, Canada",2022-04-01 00:00:00.938


In [15]:
subset_size = 100
df_subset = df.iloc[0:subset_size]

# from https://www.natasshaselvaraj.com/a-step-by-step-guide-on-geocoding-in-python/
def geocode_country(locality):
    url = 'https://nominatim.openstreetmap.org/search/' + urllib.parse.quote(locality) +'?format=json'
    response = requests.get(url).json()
    if(len(response)!=0):
        return response[0]['display_name'].split(',')[-1].strip() # takes the first result and extracts country
    else:
        return('-1')

df_subset['location'].apply(geocode_country)

df_subset.head(5)

,tweetid,hashtags,language,location,tweet_timestamp
3,1509681951116046336,[anonymous],en,International Web Zone,2022-04-01 00:00:00.337
7,1509681953053843466,"[russian, moscow]",en,India,2022-04-01 00:00:00.799
9,1509681953418711050,"[putin, medvedev, russia, ukraine]",en,"Kurdistan, Worldwide",2022-04-01 00:00:00.886
10,1509681953418752008,"[russia, putin]",en,Dubai,2022-04-01 00:00:00.886
11,1509681953636700160,"[russianarmy, chernobyl, russianukrainianwar]",en,"Ontario, Canada",2022-04-01 00:00:00.938


In [17]:
df_subset_valid_geocode = df_subset[df_subset['location'].map(lambda d: d != '-1')] # drop rows with empty location
df_subset_valid_geocode.head(5)

,tweetid,hashtags,language,location,tweet_timestamp
3,1509681951116046336,[anonymous],en,International Web Zone,2022-04-01 00:00:00.337
7,1509681953053843466,"[russian, moscow]",en,India,2022-04-01 00:00:00.799
9,1509681953418711050,"[putin, medvedev, russia, ukraine]",en,"Kurdistan, Worldwide",2022-04-01 00:00:00.886
10,1509681953418752008,"[russia, putin]",en,Dubai,2022-04-01 00:00:00.886
11,1509681953636700160,"[russianarmy, chernobyl, russianukrainianwar]",en,"Ontario, Canada",2022-04-01 00:00:00.938


In [36]:
df_subset_valid_geocode['location'].value_counts()

London, England                 5
United Kingdom                  4
Location: Right Behind You !    3
India                           2
Global                          2
                               ..
Atlanta                         1
New York City                   1
liverpool                       1
The Hague, The Netherlands      1
Austin, TX                      1
Name: location, Length: 79, dtype: int64

In [6]:
# Should ensure that locations are within list of valid 300+ countries

NameError: name 'df_subset_valid_geocode' is not defined

In [16]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 171399 entries, 3 to 364873
Data columns (total 5 columns):
 #   Column           Non-Null Count   Dtype         
---  ------           --------------   -----         
 0   tweetid          171399 non-null  int64         
 1   hashtags         171399 non-null  object        
 2   language         171399 non-null  object        
 3   location         171399 non-null  object        
 4   tweet_timestamp  171399 non-null  datetime64[ns]
dtypes: datetime64[ns](1), int64(1), object(3)
memory usage: 7.8+ MB


In [18]:
df['language'].value_counts()

en     118592
und      9020
de       8960
fr       8529
it       6655
es       6462
hi       2050
ru       1655
pl       1157
uk       1128
ja       1095
tr        831
nl        749
pt        540
ar        454
fi        297
ta        269
zh        257
sv        249
el        236
in        221
ca        195
ro        186
cs        173
th        141
da        127
et        123
lv        108
ur         86
gu         78
mr         71
bn         70
fa         56
no         49
ht         46
te         44
sl         42
iw         41
lt         41
cy         36
kn         34
ml         30
tl         30
vi         29
or         26
eu         23
ko         18
hu         18
bg         17
ne          9
ckb         9
pa          9
sr          8
my          6
ka          5
si          3
hy          2
am          2
dv          1
ps          1
Name: language, dtype: int64

In [22]:
print(df_subset['location'].values)

['International Web Zone' 'India' 'Kurdistan, Worldwide' 'Dubai'
 'Ontario, Canada' 'أخبار العالم' 'UK' 'Meganoticias Torreón'
 'Erbil, Kurdistan' 'Philadelphia, PA' 'Riyad' 'worldwide' '📍' 'Bilbao'
 'India' 'Everywhere' 'Macedonia' '福州仓山区人民公园小区1号802室'
 'Sawai Madhopur, India' 'Россия, Санкт-Петербург ' 'New Jersey, USA'
 'tokyo' 'Ethiopia ' 'Deutschland' 'Philadelphia PA USA'
 'Lost in cyberspace' '-London, Athens, Larnaca-' 'Atlanta, GA'
 'London, England' 'United Kingdom' 'The Hague, The Netherlands'
 'liverpool' 'New York City' 'United Kingdom' 'An atom in the multiverse'
 'Vastseliina, Eesti' 'United  Kingdom ' 'London, England' 'Atlanta' 'USA'
 'morocco' 'Washington DC' 'United  Kingdom ' 'Romania' 'Australia'
 'United States' 'Earth' 'Finland' 'Buenos Aires' 'Constantine, Algeria.'
 'dallas, tejas, usa, earth' 'Global' 'Warwick, PA' 'Not E.U.'
 'Vastseliina, Eesti' 'London, England' 'Florida' 'Tbilisi' 'Україна'
 'NEW JERSEY' 'An atom in the multiverse' 'London, England' '世界中'
 

In [25]:
geocode_country('London')

'United Kingdom'